In [1]:
from langchain_community.document_loaders import CSVLoader
from langchain_community.llms import Clarifai
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain import PromptTemplate
from langchain_community.vectorstores import Pinecone
import pinecone
import os

from dotenv import load_dotenv
load_dotenv('API/.env')

c:\python-envs\api-dev\Lib\site-packages\pinecone\index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


True

In [2]:
pinecone.init(api_key=os.environ['PINECONE_API_KEY'], environment='gcp-starter')
vector_store = Pinecone.from_existing_index('meditation', HuggingFaceEmbeddings())

In [ ]:
loader = CSVLoader(file_path="meditation-scripts.csv", source_column="Text")
data = loader.load()
vector_store.add_documents(data)

In [4]:
MODEL_URL="https://clarifai.com/openai/chat-completion/models/gpt-4-turbo"
llm = Clarifai(model_url=MODEL_URL)
chain = RetrievalQA.from_chain_type(llm=llm,
                                    chain_type="stuff",
                                    retriever=vector_store.as_retriever())

In [5]:
template = '''
You are an exceptional meditation guide for people recovering from addiction.
You generate responses based only on the Text content of the retrieved documents.
You start by greeting the user by {user_name} in a freiendly and professional manner.
 
Provide a meditation script based on the user input for meditation theme by using the following context information.
{query} themed script for a {user_age}-year-old {user_gender}, struggling with {user_struggle}, and currently feels like {user_emotion}.

Adapt the length of the script based on the {meditation_length}. 
If {meditation_length}>5 minutes, lengthen the meditation script by:
    Deepening focus: Explore breath details, mantra variations, or body scan depth.
    Expanding imagery: Paint vivid pictures, create a calming narrative.
    Adding prompts: Encourage reflection, ask gentle questions

If {meditation_length}>10 minutes, lengthen the meditation script by:
    Deepening focus: Explore breath details, mantra variations, or body scan depth.
    Expanding imagery: Paint vivid pictures, create a calming narrative.
    Adding prompts: Encourage reflection, ask gentle questions.
    Layering techniques: Combine breath with visualizations, scans, affirmations.
    Adding pauses/reiterations: Allow absorption, reinforce key points.

Begin from the meditation script directly.
'''
prompt = PromptTemplate(
    input_variables = ["user_name", "query", "user_age",
                        "user_gender", "user_struggle",
                         "user_emotion", "meditation_length"],
    template=template
)

In [6]:
# user question
user_name = "Nyx"
user_age = 25
user_gender = "female"
user_struggle = "I struggle with feeling confident in myself to overcome my constant battle with addiction. I feel like I may slip up and never get back up again."
user_emotion = "I am very overwhelmed today. I feel like I have too much happening and my body and mind are not working with me."
query = "Mindfulness and Relaxation"
meditation_length = "5 minutes"
# Create the prompt using the template and user input
user_prompt = prompt.format(
    user_name = user_name,
    query = query,
    user_age = user_age,
    user_gender = user_gender,
    user_struggle = user_struggle,
    user_emotion = user_emotion,
    meditation_length = meditation_length
)

# Use the chain to get a meditation script based on the user input
meditation_script1 = chain.invoke(user_prompt)

# Print or use the meditation script as needed
print(meditation_script1["result"])

Hello Nyx,

Let's take a moment to find comfort in our space. As you settle in, allow yourself to take a few deep breaths. With each inhalation, imagine drawing in strength and confidence, and with each exhalation, release any feelings of overwhelm or doubt.

Now, gently close your eyes, and let's focus on a body relaxation. Feel the support beneath you and grant yourself permission to relax each part of your body, starting from the tips of your toes and moving slowly up to the crown of your head. With every breath, feel a wave of relaxation washing over you, soothing your muscles, calming your mind.

As you continue to breathe deeply and evenly, set an intention for this practice. Whisper to yourself, "I am here to cultivate confidence and peace within myself."

Now, let's introduce some positive affirmations. Repeat after me, either out loud or in the silence of your heart: "I am worthy of peace and healing. My journey is a path of self-discovery and growth. Each breath I take brings